<a href="https://colab.research.google.com/github/rahul94jh/MSC-Research/blob/main/fvc_processing_save_as_clickbait.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We would load dataset from github directly into pandas dataframe

github repo - https://github.com/MKLab-ITI/fake-video-corpus
fvc_dup.csv - https://github.com/MKLab-ITI/fake-video-corpus/blob/master/FVC_dup.csv 

In [ ]:
!cd '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection'

In [ ]:
!pip install jsonlines

In [ ]:
#https://developers.google.com/maps/documentation/javascript/get-api-key
#https://console.cloud.google.com/apis/api/youtube.googleapis.com/overview?project=decisive-octane-311811
#https://console.cloud.google.com/apis/api/youtube.googleapis.com/credentials?project=decisive-octane-311811

api_key = 'Your_API_Key'

#Imports

In [ ]:
import os, math
import numpy as np
import pandas as pd
import requests
import shutil
from pathlib import Path
import jsonlines
from urllib.parse import urlparse, parse_qs
import json
import ast
import re

from matplotlib import pyplot as plt
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

Tensorflow version 2.5.0


# Scripts

In [ ]:
def save_df(df, path, save_index=False):
  df.to_csv(path, index=save_index)

def get_raw_data():
  local_path = '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_raw_corpus.csv'
  remote_path = 'https://raw.githubusercontent.com/MKLab-ITI/fake-video-corpus/master/FVC_dup.csv'
  if os.path.exists(local_path):
    df = pd.read_csv(local_path)
  else:
    df = pd.read_csv(remote_path)
  # save local copy 
  df = df.sample(frac=1)
  save_df(df, '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_raw_corpus.csv')

  return df

def match_youtube_url(url):
    url = url.lower() 
    return int(bool(re.match("https*://www[.]{1}youtube[.]{1}com[/]{1}\w+",url)))

def match_facebook_url(url):
    url = url.lower() 
    return int(bool(re.match("https*://www[.]{1}facebook[.]{1}com[/]{1}\w+",url)))

def extract_video_metadata(urls):
  metadata_list=[]
  for url in urls:
    url_data = urlparse(url)
    query = parse_qs(url_data.query)
    video_id = query["v"][0] 
    youtube_api_url = f'https://www.googleapis.com/youtube/v3/videos?part=snippet&id={video_id}&key={api_key}'
    r = requests.get(youtube_api_url, stream = True)      
    if r.status_code == 200:
       r.raw.decode_content = True
       data = r.json()
       if len(data['items'])>0:
         metadata_list.append(json.dumps(r.json(), ensure_ascii=False))
    else:
      print(f'failled..{r.status_code}')
  return np.array(metadata_list)

def write_json_data(path, data):
    with open(path,'w') as f:
        f.writelines("%s\n" % line for line in data)

def preprocess_string(text):
    text_cleaned = re.sub(r"http\S+", "", text, flags=re.IGNORECASE|re.MULTILINE)  # remove any url in the text
    text_cleaned = re.sub(r"\w+@\w{1,}\.\w{1,}","", text_cleaned, flags=re.IGNORECASE|re.MULTILINE) # remove emails from text
    text_cleaned = re.sub(r'[^a-z\s]+',' ',text_cleaned,flags=re.IGNORECASE|re.MULTILINE) # remove non alphabetics char
    text_cleaned = re.sub(r'(\s+)',' ',text_cleaned, flags=re.IGNORECASE|re.MULTILINE)  # remove spaces

    text_cleaned = text_cleaned.lstrip()
    text_cleaned = text_cleaned.rstrip()
    text_cleaned = text_cleaned.lower() 

    return text_cleaned 

def extract_list_data(row, root, column_name, sub_column='', clean_text=True):
  items = row[root][0]
  if len(sub_column)>0:
    if sub_column in items[column_name]:
      text = items[column_name][sub_column]
    else:
      text='NA'
  else:
    text = items[column_name]
  
  if clean_text:
      text = preprocess_string(text)

  return text

def extract_from_dict(row, column, subcolumn):
  return row[column][subcolumn]


def download_save_image(df):
  df_new = df.copy(deep=True)

  for i in range(len(df_new)):
    row = df_new.iloc[i]

    id = row['id']  
    url = row['thumbnail_320_180'] 
    is_fake = row['is_fake'] 


    filename = f'{id}' + '_' + url.split("/")[-1]      

    if is_fake:      
        image_path = os.path.join(clickbaitImage_dir,filename) 
    else:
        image_path = os.path.join(nonclickbaitsImage_dir,filename) 

     
    if not os.path.exists(image_path):                       
            r = requests.get(url, stream = True)             
            
            if r.status_code == 200:                         
                r.raw.decode_content = True

                with open(image_path,'wb') as f:
                    shutil.copyfileobj(r.raw, f)             

                print(i, ': success')

                df_new.loc[(df_new['id'] == id), 'thumbnail_320_180_path'] = image_path        

            else:
                print(i, ': failed')

                df_new.loc[(df_new['id'] == id), 'thumbnail_320_180_path'] = '?'                  

    else:
        df_new.loc[(df_new['id'] == id), 'thumbnail_320_180_path'] = image_path 
        print('file exist', image_path)

  return df_new 


#Load Data

In [ ]:
df_raw = get_raw_data()
df_raw.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5199 entries, 2205 to 1712
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cascade_id  5199 non-null   object
 1   video_url   5199 non-null   object
 2   label       5199 non-null   object
dtypes: object(3)
memory usage: 162.5+ KB


In [ ]:
df_raw.sample(10)

,cascade_id,video_url,label
572,r172,https://www.youtube.com/watch?v=t_P7emQGz6A,real
670,f93,https://www.facebook.com/hcvideos/videos/46205...,fake
2756,f96,https://www.youtube.com/watch?v=JioGUnk8p6I,fake
809,r5,https://www.youtube.com/watch?v=Qr8uqmskQm8,real
3875,r33,https://www.youtube.com/watch?v=QZtgXKWv59M,real
1176,f54,https://www.youtube.com/watch?v=p5OOxLoth0U,debunked
794,r27,https://www.youtube.com/watch?v=zme8S9sym3E,real
4900,r21,https://www.facebook.com/MQFofficial/videos/17...,real
4077,f54,https://www.youtube.com/watch?v=NhkGkGELAg0,fake
3781,r165,https://www.facebook.com/colima.digitall/video...,real


# Preprocessing

### General

The video url contains url other than of youtube as well, here we will focus only on fake video on youtube hence we would be filtering out the video url for youtube and discarding others using the regular expression string match. The below link is the visualization for the regex we would be using.

https://regexper.com/#https*%3A%5B%2F%2F%5Dwww%5B.%5D%7B1%7Dyoutube%5B.%5D%7B1%7Dcom%5B%2F%5D%7B1%7D%5Cw%2B 

In [ ]:
df_raw['is_youtube_link'] = df_raw['video_url'].apply(lambda x : match_youtube_url(x))
df_raw['is_facebook_link'] = df_raw['video_url'].apply(lambda x : match_facebook_url(x))

df_raw.sample(10)

,cascade_id,video_url,label,is_youtube_link,is_facebook_link
4088,r117,https://www.facebook.com/novostiATO/videos/850...,real,0,1
2808,f166,https://www.facebook.com/UnionDesPagesScientif...,fake,0,1
3795,f93,https://www.facebook.com/OfficialPandaNetwork/...,uncertain,0,1
738,r46,https://www.youtube.com/watch?v=fnv6zdrS8PM,real,1,0
2774,f155,https://www.facebook.com/www.KalashnikovCorpor...,fake,0,1
215,r107,https://www.facebook.com/macropolisqr/videos/1...,real,0,1
4131,f155,https://www.youtube.com/watch?v=ORWluunJ_84,fake,1,0
4225,f93,https://www.youtube.com/watch?v=j7nRiQh9Ew8,fake,1,0
3083,f97,https://www.youtube.com/watch?v=d-0rM0ZwJfA,fake,1,0
848,f84,https://www.youtube.com/watch?v=_W_t4KYcblA,fake,1,0


In [ ]:
df_raw.is_youtube_link.value_counts()

1    3028
0    2171
Name: is_youtube_link, dtype: int64

In [ ]:
df_raw.is_facebook_link.value_counts()

0    3169
1    2030
Name: is_facebook_link, dtype: int64

In [ ]:
# Though we wouldn't use facebook related rows but its always good to save each form of data
save_df(df_raw, '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_raw_pre_1.csv')

In [ ]:
# Filter for YouTube only rows
df_filtered = df_raw.loc[(df_raw.loc[:,'is_youtube_link']==1),['video_url', 'label']]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3028 entries, 545 to 1712
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   video_url  3028 non-null   object
 1   label      3028 non-null   object
dtypes: object(2)
memory usage: 71.0+ KB


In [ ]:
df_filtered.label.value_counts()

fake         1679
real         1015
uncertain     207
debunked       70
parody         57
Name: label, dtype: int64

In [ ]:
# We are only interested in rows with label fake or real
df_filtered = df_filtered.loc[(df_filtered['label'].isin(['fake','real']))]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2694 entries, 545 to 1712
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   video_url  2694 non-null   object
 1   label      2694 non-null   object
dtypes: object(2)
memory usage: 63.1+ KB


In [ ]:
# Convert label to numeric value -> fake is 1 and real is 0
df_filtered.label = df_filtered.label.astype('category') 
df_filtered.label = df_filtered.label.cat.codes
df_filtered.label.value_counts()

0    1679
1    1015
Name: label, dtype: int64

In [ ]:
#Shuffle the dataframe for randomness
df_filtered_new = df_filtered.sample(frac=1).reset_index(drop=True)
df_filtered_new.head()

,video_url,label
0,https://www.youtube.com/watch?v=FG1LGKieTxY,1
1,https://www.youtube.com/watch?v=PdfwDNjMruY,0
2,https://www.youtube.com/watch?v=DtTNkeXG9Pc,1
3,https://www.youtube.com/watch?v=8-WacNYXZHU,0
4,https://www.youtube.com/watch?v=Q_aoEvZcxr0,1


In [ ]:
save_df(df_filtered, '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_youtube_only.csv')

### Download Metdata using youtube data api




In [ ]:
fakes = np.array(df_filtered.loc[(df_filtered.label==0), ['video_url']]).reshape(-1)
reals = np.array(df_filtered.loc[(df_filtered.label==1), ['video_url']]).reshape(-1)

df_filtered.shape, fakes.shape, reals.shape

((2694, 2), (1679,), (1015,))

In [ ]:
# YouTube api returns json data for each video, hence create two files to copy json data
fvc_fakes_metadata = Path('/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_fakes_metadata.json')
fvc_reals_metadata = Path('/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_reals_metadata.json')

fvc_fakes_metadata.touch(exist_ok=True)
fvc_reals_metadata.touch(exist_ok=True)

We would be using YouTube data api v3 for fetching the metadata of videos from video url such as video title, descriptions, thumbnails e.t.c. Get the api key, follow below links for obtaining api key. Learn more about YouTube data api from below official websites

https://developers.google.com/youtube/v3/docs/?apix=true 

https://developers.google.com/maps/documentation/javascript/get-api-key
https://console.cloud.google.com/marketplace/product/google/youtube.googleapis.com?q=search&referrer=search&project=decisive-octane-311811

In [ ]:
# Download metadata for fake videos from YouTube
fake_videos_metadata = extract_video_metadata(fakes)

In [ ]:
# # Download metadata for real videos from YouTube
real_videos_metadata = extract_video_metadata(reals)

In [ ]:
# We had almost 2600 videos URL's but we can see that the metedata for only around 1800 videos we could fetch. 
fake_videos_metadata.shape, real_videos_metadata.shape

((1104,), (705,))

In [ ]:
# Let's write the downloaded data to our local json file
write_json_data(fvc_fakes_metadata, fake_videos_metadata)
write_json_data(fvc_reals_metadata, real_videos_metadata)

### Convert metadata to csv from JSON

In [ ]:
fake_data_dict_array = []
real_data_dict_array = []
with jsonlines.open(fvc_fakes_metadata) as reader:
    for obj in reader:
      fake_data_dict_array.append(obj)

with jsonlines.open(fvc_reals_metadata) as reader:
    for obj in reader:
      real_data_dict_array.append(obj)

len(real_data_dict_array), len(fake_data_dict_array)

(705, 1104)

In [ ]:
fake_data_dict_array[0]['items'][0]['snippet']['title']

'UFO spotted in Sao Paulo?'

In [ ]:
df_real_metadata = pd.DataFrame.from_dict(real_data_dict_array)
df_fake_metadata = pd.DataFrame.from_dict(fake_data_dict_array)
df_fake_metadata.shape, df_real_metadata.shape

((1104, 4), (705, 4))

In [ ]:
df_real_metadata['is_fake'] = 0
df_fake_metadata['is_fake'] = 1

FVC_df = pd.concat([df_real_metadata, df_fake_metadata], ignore_index=True)
FVC_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1809 entries, 0 to 1808
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   kind      1809 non-null   object
 1   etag      1809 non-null   object
 2   items     1809 non-null   object
 3   pageInfo  1809 non-null   object
 4   is_fake   1809 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 70.8+ KB


In [ ]:
FVC_df = FVC_df.sample(frac=1).reset_index(drop=True)
FVC_df.head(10)

,kind,etag,items,pageInfo,is_fake
0,youtube#videoListResponse,qXgJ4GBHFacYL_ilki8XHQgzkOk,"[{'kind': 'youtube#video', 'etag': '1RbaajChwA...","{'totalResults': 1, 'resultsPerPage': 1}",1
1,youtube#videoListResponse,3FlLC8kAQ4aQyVGMYGDHRuZnEdg,"[{'kind': 'youtube#video', 'etag': 'tz2Gg6j9Y7...","{'totalResults': 1, 'resultsPerPage': 1}",0
2,youtube#videoListResponse,jEslIxG66jM1PHPxNMmu6NpvgjE,"[{'kind': 'youtube#video', 'etag': 'fBwrp0mb9X...","{'totalResults': 1, 'resultsPerPage': 1}",0
3,youtube#videoListResponse,pQfbko7mQ6iZQtaP9VR5y4TsYKQ,"[{'kind': 'youtube#video', 'etag': 'DOyfwLNM3Q...","{'totalResults': 1, 'resultsPerPage': 1}",0
4,youtube#videoListResponse,QiBMyZmJ0o-nVpipkGx8xI3L71E,"[{'kind': 'youtube#video', 'etag': 'cexpvnjrro...","{'totalResults': 1, 'resultsPerPage': 1}",1
5,youtube#videoListResponse,jSJ8lpLoKXIevhjnmIpbXgUQ5eY,"[{'kind': 'youtube#video', 'etag': 'uz__8uwli0...","{'totalResults': 1, 'resultsPerPage': 1}",1
6,youtube#videoListResponse,PzYpIje8-UZFcekvBZUOycQ_x08,"[{'kind': 'youtube#video', 'etag': 'yUYj1Zpzvz...","{'totalResults': 1, 'resultsPerPage': 1}",1
7,youtube#videoListResponse,hbBn2rPVmN7_xx7SCwvVpUhgKN0,"[{'kind': 'youtube#video', 'etag': '7afUx5wz64...","{'totalResults': 1, 'resultsPerPage': 1}",0
8,youtube#videoListResponse,wzblsK3hSVc8KQBvasjnKWtt9rs,"[{'kind': 'youtube#video', 'etag': '1AZpRHx_dt...","{'totalResults': 1, 'resultsPerPage': 1}",1
9,youtube#videoListResponse,v48iBYTbiDPWDHMOZ0SJOsNX8PI,"[{'kind': 'youtube#video', 'etag': 'lIQYJ8SuPY...","{'totalResults': 1, 'resultsPerPage': 1}",0


In [ ]:
save_df(FVC_df, '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_with_metadata_raw.csv')

## Preprocess FVC metadata

In [ ]:
FVC_metadata_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_with_metadata_raw.csv')
FVC_metadata_df = FVC_df.loc[:,['items','is_fake']]
FVC_metadata_df.head()

,items,is_fake
0,"[{'kind': 'youtube#video', 'etag': '1RbaajChwA...",1
1,"[{'kind': 'youtube#video', 'etag': 'tz2Gg6j9Y7...",0
2,"[{'kind': 'youtube#video', 'etag': 'fBwrp0mb9X...",0
3,"[{'kind': 'youtube#video', 'etag': 'DOyfwLNM3Q...",0
4,"[{'kind': 'youtube#video', 'etag': 'cexpvnjrro...",1


In [ ]:
FVC_metadata_df['id'] = FVC_metadata_df.apply(lambda x : extract_list_data(x,'items','id', clean_text=False),axis=1)
FVC_metadata_df['title'] = FVC_metadata_df.apply(lambda x : extract_list_data(x,'items','snippet','title'), axis=1)
FVC_metadata_df['description'] = FVC_metadata_df.apply(lambda x : extract_list_data(x,'items','snippet','description'), axis=1)
FVC_metadata_df['tags'] = FVC_metadata_df.apply(lambda x : extract_list_data(x,'items','snippet','tags', clean_text=False), axis=1)
FVC_metadata_df['thumbnails'] = FVC_metadata_df.apply(lambda x : extract_list_data(x,'items','snippet','thumbnails', clean_text=False), axis=1)
FVC_metadata_df['thumbnail_320_180'] = FVC_metadata_df.loc[:,'thumbnails'].apply(lambda x : extract_from_dict(x,'medium','url'))
FVC_metadata_df['thumbnail_120_90'] = FVC_metadata_df.loc[:,'thumbnails'].apply(lambda x : extract_from_dict(x,'default','url'))
FVC_metadata_df['thumbnail_480_360'] = FVC_metadata_df.loc[:,'thumbnails'].apply(lambda x : extract_from_dict(x,'high','url'))
FVC_metadata_df = FVC_metadata_df.drop(['items','thumbnails'], axis=1)

In [ ]:
# Remove rows where both title and description are not present
FVC_metadata_df = FVC_metadata_df.loc[~((FVC_metadata_df.description == '') & (FVC_metadata_df.title == ''))]
FVC_metadata_df.shape

(1589, 8)

In [ ]:
save_df(FVC_metadata_df, '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_with_metadata_processed.csv')

## Download and map thumbnails

In [ ]:
FVC_filtered = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_with_metadata_processed.csv')

In [ ]:
clickbaitImage_dir = '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/thumbnails/clickbaits'
nonclickbaitsImage_dir = '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/thumbnails/nonclickbaits'

if not os.path.exists(clickbaitImage_dir):
    os.makedirs(clickbaitImage_dir)

if not os.path.exists(nonclickbaitsImage_dir):
    os.makedirs(nonclickbaitsImage_dir)

In [ ]:
#Download thumbnails and map the downloaded file path
FVC_filtered_mapped = download_save_image(FVC_filtered)

0 : success
1 : success
2 : success
3 : success
4 : success
5 : success
6 : success
7 : success
8 : success
9 : success
10 : success
11 : success
12 : success
13 : success
14 : success
15 : success
16 : success
17 : success
18 : success
19 : success
20 : success
21 : success
22 : success
23 : success
24 : success
25 : success
26 : success
27 : success
28 : success
29 : success
30 : success
31 : success
32 : success
33 : success
34 : success
35 : success
36 : success
37 : success
38 : success
39 : success
40 : success
41 : success
42 : success
43 : success
44 : success
45 : success
46 : success
47 : success
48 : success
49 : success
50 : success
51 : success
52 : success
53 : success
54 : success
55 : success
56 : success
57 : success
58 : success
59 : success
60 : success
61 : success
62 : success
63 : success
64 : success
65 : success
66 : success
67 : success
68 : success
69 : success
70 : success
71 : success
72 : success
73 : success
74 : success
75 : success
76 : success
77 : succ

In [ ]:
FVC_filtered_mapped = FVC_filtered_mapped.rename(columns = {'is_fake': 'is_clickbait'}, inplace = False)

Index(['is_clickbait', 'id', 'title', 'description', 'tags',
       'thumbnail_320_180', 'thumbnail_120_90', 'thumbnail_480_360',
       'thumbnail_320_180_path'],
      dtype='object')

In [ ]:
save_df(FVC_filtered_mapped, '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_with_metadata_processed.csv')

## Prepare dataset for clickbait detection

In [ ]:
FVC_clickbait_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/fvc_with_metadata_processed.csv')

In [ ]:
FVC_clickbait_df = FVC_clickbait_df.drop(['id','tags','thumbnail_120_90','thumbnail_320_180', 'thumbnail_480_360'],axis=1)
FVC_clickbait_df.columns

Index(['is_clickbait', 'title', 'description', 'thumbnail_320_180_path'], dtype='object')

In [ ]:
# check for bad data
FVC_clickbait_df.loc[FVC_clickbait_df['thumbnail_320_180_path']=='?']

,is_clickbait,title,description,thumbnail_320_180_path


In [ ]:
FVC_clickbait_df = FVC_clickbait_df.fillna('NA')

In [ ]:
def get_combined_text(row):
  if row['title'] != 'NA' and row['description'] !='NA' :
    return row['title']+' '+row['description']
  elif row['title'] != 'NA':
    return row['title']
  else:
    return row['description']

In [ ]:
FVC_clickbait_df['text'] = FVC_clickbait_df.apply(lambda x: get_combined_text(x), axis=1)
FVC_clickbait_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1589 entries, 0 to 1588
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   is_clickbait            1589 non-null   int64 
 1   title                   1589 non-null   object
 2   description             1589 non-null   object
 3   thumbnail_320_180_path  1589 non-null   object
 4   text                    1589 non-null   object
dtypes: int64(1), object(4)
memory usage: 62.2+ KB


In [ ]:
FVC_clickbait_df = FVC_clickbait_df.drop(['title','description'], axis=1)
FVC_clickbait_df.columns

Index(['is_clickbait', 'thumbnail_320_180_path', 'text'], dtype='object')

In [ ]:
FVC_clickbait_df.is_clickbait.value_counts()

1    930
0    659
Name: is_clickbait, dtype: int64

In [ ]:
FVC_clickbait_df = FVC_clickbait_df.sample(frac=1)
FVC_clickbait_df.shape

(1589, 3)

In [ ]:
save_df(FVC_clickbait_df, '/content/drive/MyDrive/Colab Notebooks/clcikbait_detection/dataset/FVC_CORPUS/FVC_clickbait.csv')